# Bygg en regressionsmodell: förbered och visualisera data

## **Linjär regression för pumpor - Lektion 2**
#### Introduktion

Nu när du har verktygen du behöver för att börja bygga maskininlärningsmodeller med Tidymodels och Tidyverse, är du redo att börja ställa frågor till dina data. När du arbetar med data och tillämpar ML-lösningar är det mycket viktigt att förstå hur man ställer rätt fråga för att verkligen utnyttja potentialen i din dataset.

I den här lektionen kommer du att lära dig:

- Hur du förbereder dina data för modellbyggande.

- Hur du använder `ggplot2` för datavisualisering.

Frågan du behöver svar på avgör vilken typ av ML-algoritmer du kommer att använda. Och kvaliteten på svaret du får tillbaka kommer att vara starkt beroende av datans natur.

Låt oss se detta genom att arbeta igenom en praktisk övning.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Konstverk av @allison_horst</figcaption>


<!--![Konstverk av \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Konstverk av \@allison_horst-->


## 1. Importera pumpadata och kalla på Tidyverse

Vi kommer att behöva följande paket för att hantera denna lektion:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) är en [samling av R-paket](https://www.tidyverse.org/packages) som är utformade för att göra datavetenskap snabbare, enklare och roligare!

Du kan installera dem med:

`install.packages(c("tidyverse"))`

Skriptet nedan kontrollerar om du har de paket som krävs för att slutföra denna modul och installerar dem åt dig om några saknas.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Nu sätter vi igång några paket och laddar [data](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) som tillhandahålls för denna lektion!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

En snabb `glimpse()` visar direkt att det finns tomma värden och en blandning av strängar (`chr`) och numeriska data (`dbl`). `Date` är av typen tecken och det finns också en märklig kolumn kallad `Package` där datan är en blandning av `sacks`, `bins` och andra värden. Datan är faktiskt lite av en röra 😤.

Det är faktiskt inte särskilt vanligt att få en dataset som är helt redo att användas för att skapa en ML-modell direkt. Men oroa dig inte, i denna lektion kommer du att lära dig hur man förbereder en rå dataset med hjälp av standardbibliotek i R 🧑‍🔧. Du kommer också att lära dig olika tekniker för att visualisera datan.📈📊
<br>

> En påminnelse: Pipe-operatorn (`%>%`) utför operationer i logisk sekvens genom att skicka ett objekt framåt in i en funktion eller ett uttryck. Du kan tänka på pipe-operatorn som att säga "och sedan" i din kod.


## 2. Kontrollera för saknade data

Ett av de vanligaste problemen som dataanalytiker måste hantera är ofullständig eller saknad data. R representerar saknade eller okända värden med ett speciellt sentinelvärde: `NA` (Not Available).

Så hur kan vi veta att dataframen innehåller saknade värden?
<br>
-   Ett direkt sätt skulle vara att använda R:s inbyggda funktion `anyNA`, som returnerar de logiska objekten `TRUE` eller `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Bra, det verkar som att det saknas viss data! Det är en bra utgångspunkt.

-   Ett annat sätt skulle vara att använda funktionen `is.na()` som visar vilka enskilda kolumnelement som saknas med ett logiskt värde `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Okej, jobbet är klart men med en så stor dataram som denna skulle det vara ineffektivt och praktiskt taget omöjligt att granska alla rader och kolumner individuellt😴.

-   Ett mer intuitivt sätt skulle vara att beräkna summan av de saknade värdena för varje kolumn:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Mycket bättre! Det saknas data, men kanske spelar det ingen roll för uppgiften. Låt oss se vad vidare analys ger.

> Tillsammans med de fantastiska uppsättningarna av paket och funktioner har R en mycket bra dokumentation. Till exempel, använd `help(colSums)` eller `?colSums` för att ta reda på mer om funktionen.


## 3. Dplyr: En grammatik för datamanipulation

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Konstverk av @allison_horst</figcaption>


<!--![Konstverk av \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Konstverk av \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), ett paket i Tidyverse, är en grammatik för datamanipulation som erbjuder en konsekvent uppsättning verb som hjälper dig att lösa de vanligaste utmaningarna inom datamanipulation. I det här avsnittet ska vi utforska några av dplyrs verb!  
<br>


#### dplyr::select()

`select()` är en funktion i paketet `dplyr` som hjälper dig att välja vilka kolumner du vill behålla eller exkludera.

För att göra din data frame enklare att arbeta med, ta bort flera av dess kolumner med hjälp av `select()` och behåll endast de kolumner du behöver.

Till exempel, i denna övning kommer vår analys att involvera kolumnerna `Package`, `Low Price`, `High Price` och `Date`. Låt oss välja dessa kolumner.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` är en funktion i paketet `dplyr` som hjälper dig att skapa eller ändra kolumner, samtidigt som de befintliga kolumnerna behålls.

Den generella strukturen för mutate är:

`data %>%   mutate(new_column_name = what_it_contains)`

Låt oss testa `mutate` med hjälp av kolumnen `Date` genom att utföra följande operationer:

1.  Konvertera datumen (som för närvarande är av typen tecken) till ett månadsformat (det här är amerikanska datum, så formatet är `MM/DD/YYYY`).

2.  Extrahera månaden från datumen till en ny kolumn.

I R gör paketet [lubridate](https://lubridate.tidyverse.org/) det enklare att arbeta med datum- och tidsdata. Så låt oss använda `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` och se hur vi kan uppnå ovanstående mål. Vi kan ta bort kolumnen Date eftersom vi inte kommer att behöva den igen i efterföljande operationer.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Nu ska vi skapa en ny kolumn `Price`, som representerar det genomsnittliga priset på en pumpa. Låt oss nu ta genomsnittet av kolumnerna `Low Price` och `High Price` för att fylla i den nya kolumnen Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Jaaa!💪

"Men vänta lite!", säger du efter att ha skummat igenom hela datasettet med `View(pumpkins)`, "Det är något konstigt här!"🤔

Om du tittar på kolumnen `Package`, säljs pumpor i många olika konfigurationer. Vissa säljs i mått som `1 1/9 bushel`, andra i `1/2 bushel`, några per pumpa, några per pund, och vissa i stora lådor med varierande bredder.

Låt oss kontrollera detta:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Fantastiskt!👏

Pumpor verkar vara väldigt svåra att väga konsekvent, så låt oss filtrera dem genom att välja endast pumpor med strängen *bushel* i kolumnen `Package` och lägga detta i en ny data frame `new_pumpkins`.


#### dplyr::filter() och stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): skapar en delmängd av data som endast innehåller **rader** som uppfyller dina villkor, i detta fall pumpor med strängen *bushel* i kolumnen `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): identifierar förekomsten eller frånvaron av ett mönster i en sträng.

Paketet [`stringr`](https://github.com/tidyverse/stringr) erbjuder enkla funktioner för vanliga strängoperationer.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Du kan se att vi har begränsat oss till ungefär 415 rader med data som innehåller pumpor i skäppor.🤩  
<br>


#### dplyr::case_when()

**Men vänta! Det finns en sak till att göra**

Märkte du att mängden per skäppa varierar per rad? Du behöver normalisera prissättningen så att du visar priset per skäppa, inte per 1 1/9 eller 1/2 skäppa. Dags att göra lite matematik för att standardisera det.

Vi kommer att använda funktionen [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) för att *mutera* kolumnen Price beroende på vissa villkor. `case_when` gör det möjligt att vektorisera flera `if_else()`-satser.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nu kan vi analysera priset per enhet baserat på deras mått i bushels. All denna undersökning av pumpors bushels visar dock hur `viktigt` det är att `förstå din datas natur`!

> ✅ Enligt [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) beror en bushels vikt på typen av gröda, eftersom det är en volymmätning. "En bushel tomater, till exempel, ska väga 56 pund... Blad och gröna tar upp mer plats med mindre vikt, så en bushel spenat väger bara 20 pund." Det är ganska komplicerat! Låt oss inte bry oss om att göra en omvandling från bushel till pund, utan istället prissätta per bushel. All denna undersökning av pumpors bushels visar dock hur viktigt det är att förstå din datas natur!
>
> ✅ Lade du märke till att pumpor som säljs per halv-bushel är väldigt dyra? Kan du lista ut varför? Tips: små pumpor är mycket dyrare än stora, antagligen eftersom det finns så många fler av dem per bushel, med tanke på det outnyttjade utrymmet som en stor ihålig pajpumpa tar upp.


Nu slutligen, för äventyrets skull 💁‍♀️, låt oss också flytta kolumnen Månad till första positionen, dvs `före` kolumnen `Paket`.

`dplyr::relocate()` används för att ändra kolumnpositioner.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Bra jobbat! 👌 Du har nu ett rent och prydligt dataset som du kan använda för att bygga din nya regressionsmodell!  
<br>


## 4. Datavisualisering med ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografik av Dasani Madipalli</figcaption>


<!--![Infografik av Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Det finns ett *klokt* ordspråk som lyder så här:

> "Den enkla grafen har fört mer information till dataanalytikerns sinne än någon annan metod." --- John Tukey

En del av dataforskarens roll är att visa kvaliteten och karaktären hos de data de arbetar med. För att göra detta skapar de ofta intressanta visualiseringar, eller diagram, grafer och tabeller, som visar olika aspekter av data. På så sätt kan de visuellt visa relationer och luckor som annars är svåra att upptäcka.

Visualiseringar kan också hjälpa till att avgöra vilken maskininlärningsteknik som är mest lämplig för datan. Ett spridningsdiagram som verkar följa en linje, till exempel, indikerar att datan är en bra kandidat för en linjär regressionsanalys.

R erbjuder flera system för att skapa grafer, men [`ggplot2`](https://ggplot2.tidyverse.org/index.html) är ett av de mest eleganta och mångsidiga. `ggplot2` gör det möjligt att skapa grafer genom att **kombinera oberoende komponenter**.

Låt oss börja med ett enkelt spridningsdiagram för kolumnerna Price och Month.

I det här fallet börjar vi med [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), tillhandahåller en dataset och estetisk mappning (med [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) och lägger sedan till lager (som [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) för spridningsdiagram.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Är detta en användbar graf 🤷? Är det något med den som förvånar dig?

Den är inte särskilt användbar eftersom allt den gör är att visa dina data som en spridning av punkter under en viss månad.
<br>


### **Hur gör vi det användbart?**

För att få diagram att visa användbar data behöver du oftast gruppera datan på något sätt. Till exempel, i vårt fall skulle det ge mer insikt i de underliggande mönstren i vår data om vi hittar det genomsnittliga priset på pumpor för varje månad. Detta leder oss till ännu en snabbgenomgång av **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Grupperad aggregering i R kan enkelt beräknas med

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` ändrar analysenheten från hela datasetet till individuella grupper, som per månad.

-   `dplyr::summarize()` skapar en ny dataram med en kolumn för varje grupperingsvariabel och en kolumn för varje sammanfattningsstatistik som du har specificerat.

Till exempel kan vi använda `dplyr::group_by() %>% summarize()` för att gruppera pumporna i grupper baserade på **Month**-kolumnen och sedan hitta **medelpriset** för varje månad.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Kategoriska funktioner, såsom månader, representeras bäst med ett stapeldiagram 📊. De lager som används för stapeldiagram är `geom_bar()` och `geom_col()`. Konsultera `?geom_bar` för att lära dig mer.

Låt oss skapa ett!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Det här är en mer användbar datavisualisering! Det verkar indikera att det högsta priset för pumpor inträffar i september och oktober. Stämmer det med dina förväntningar? Varför eller varför inte?

Grattis till att ha avslutat den andra lektionen 👏! Du förberedde dina data för modellbyggande och upptäckte sedan fler insikter med hjälp av visualiseringar!



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, vänligen notera att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på sitt ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
